# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-10-11&until-occurred-date=2021-10-18


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

14181 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-11&until-occurred-date=2021-10-18
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=2ccbe6d7-a09b-4a41-8174-c0af31c01d24&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-11&until-occurred-date=2021-10-18
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=fc2d57ed-7929-4a82-ada2-63fc8a23bbb6&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-10-11&until-occurred-date=2021-10-18
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-07-29.csv
failed to load preprint_tweets_2021-09-06.csv
failed to load preprint_tweets_2021-09-13.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-10-05.csv
failed to load preprint_tweets_2021-10-11.csv
failed to load preprint_tweets_2021-10-04.csv
failed to load preprint_tweets_2021-10-06.csv
failed to load preprint_tweets_2021-10-07.csv
failed to load preprint_tweets_2021-07-26.csv
failed to load preprint_tweets_2021-09-20.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.08.24.21262415...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264260...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264262...
REST API query complete  200
REST API query started for 10.1101/2021.09.13.21262182...
REST API query complete  200
REST API query started for 10.1101/2021.09.22.21263977...
REST API query complete  200
REST API query started for 10.1101/2021.02.16.21251535...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.09.30.462488...
REST API query complete  200
REST API query started for 10.1101/2021.08.19.21262139...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.07.08

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.10.11.21264863,226,medRxiv,Obstetrics and Gynecology,True,Characterizing menstrual bleeding changes occu...,"[{'name': 'Katharine MN Lee'}, {'name': 'Elean...",<p>Many people began sharing that they experie...,2021-10-12
1,10.1101/2021.10.10.463851,95,bioRxiv,Evolutionary Biology,False,The Japanese wolf is most closely related to m...,"[{'name': 'Jun Gojobori'}, {'name': 'Nami Arak...",<p>The Japanese wolf (<italic>Canis lupus hodo...,2021-10-11
2,10.1101/2021.10.16.464636,88,bioRxiv,Plant Biology,False,Automatic identification and annotation of MYB...,[{'name': 'Boas Pucker'}],<p>Background: MYBs are among the largest tran...,2021-10-16
3,10.1101/2021.10.12.464039,72,bioRxiv,Molecular Biology,False,Histone H1 regulates non-coding RNA turnover o...,"[{'name': 'José Miguel Fernández-Justel'}, {'n...",<p>Linker histones are highly abundant chromat...,2021-10-12
4,10.1101/2021.10.08.463334,44,bioRxiv,Genomics,True,Covid-19 genomic analysis reveals clusters of ...,[{'name': 'Evans K. Rono'}],<p>The emerging SARS-CoV-2 variants may potent...,2021-10-12
5,10.1101/2021.10.14.464465,42,bioRxiv,Bioinformatics,False,Polypolish: short-read polishing of long-read ...,"[{'name': 'Ryan R Wick'}, {'name': 'Kathryn E ...",<p>Long-read-only bacterial genome assemblies ...,2021-10-16
6,10.1101/2021.10.12.464066,41,bioRxiv,Cell Biology,False,Tissue-wide coordination of calcium signaling ...,"[{'name': 'Jessica L Moore'}, {'name': 'Feng G...",<p>Regenerative processes in the mammalian ski...,2021-10-13
7,10.1101/2021.10.12.463965,39,bioRxiv,Cell Biology,False,Ferroptosis Regulation by the NGLY1/NFE2L1 Pat...,"[{'name': 'Giovanni C. Forcina'}, {'name': 'La...",<p>Ferroptosis is an oxidative form of non-apo...,2021-10-12
8,10.1101/2021.10.13.21264960,36,medRxiv,Infectious Diseases (except HIV/AIDS),True,Evaluation of real-life use of Point-Of-Care R...,"[{'name': 'Ana C. Blanchard'}, {'name': 'Marc ...",<p>We evaluated the use of rapid antigen detec...,2021-10-14
9,10.1101/2021.10.10.21264827,36,medRxiv,Infectious Diseases (except HIV/AIDS),True,Heterologous SARS-CoV-2 Booster Vaccinations –...,"[{'name': 'Robert L. Atmar'}, {'name': 'Kirste...",<sec><title>Background</title><p>While Coronav...,2021-10-13


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

59 preprints (including 9 covering SARS-CoV-2, 52 from bioRxiv and 7 from medRxiv) published in the last 7 days before 2021-10-18 had been tweeted at least 3 times (maximum 226).